# Face Detection with Amazon SageMaker

This notebook provides an overview at how Amazon SageMaker effortlessly simplifies the machine learning development cycle for even the most complex of models, such as the Single Shot Detector (SSD) for Face Detection.

Amazon SageMaker is a fully-managed service that simplifies the process of **building**, **training**, and **deploying** machine learning and deep learning models at scale. It consists of three components:

* **Build**: A fully managed Jupyter Notebook instance for loading data, prototyping models, and launching training jobs and deployment endpoints. This comes pre-configured with all of the standard deep learning libraries for both CPU and GPU computation. 
* **Train**: With one-click, SageMaker launches fully managed, scalable training jobs on AWS for both proprietary SageMaker ML algorithms and custom-designed deep learning architectures.
* **Deploy**: With one-click SageMaker provides a fully managed service for deploying trained models to endpoint instances to host model inference as an API call.

### Set-up
#### Imports

In [ ]:
import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.mxnet import MXNet, MXNetModel
import matplotlib.pyplot as plt
import pickle
import cv2
import random
import numpy as np
from PIL import Image
import urllib
%matplotlib inline

#### Downloads

In [ ]:
!aws s3 cp s3://sagemaker-obj-det-demo/mxnet_ssd_face_detection.py .
!aws s3 cp s3://sagemaker-obj-det-demo/1_Handshaking_Handshaking_1_203.jpg .
!aws s3 cp s3://sagemaker-obj-det-demo/19_Couple_Couple_19_187.jpg .

## Build

The SageMaker Notebook instance is a convenient platform to explore data for a given task and preprocess if necessary.
We can use this environment to easily launch training jobs using this data, and deploy trained models for inference.

### Data Exploration

Our model is tasked with Face Detection. Let's explore the data provided to us to understand how such a model might work.

The data set the model will be trained on is a set of images:

In [ ]:
img_array = np.array(Image.open('1_Handshaking_Handshaking_1_203.jpg'))
plt.imshow(img_array)

With a corresponding set of numbers:

In [ ]:
# 1--Handshaking/1_Handshaking_Handshaking_1_203.jpg
# 4
# 142.756032 258.058981 112.557641 216.879357
# 241.587131 433.758713 90.595174 161.973190
# 691.817694 387.088472 82.359249 178.445040
# 782.412869 527.099196 85.104558 175.699732

The numbers here describe bounding boxes for the four faces in the image.

Each row represents: <tt>left top width height</tt>. Let's put these together with the input:

In [ ]:
boxes = [
         [int(np.round(142.756032)), int(np.round(258.058981)), int(np.round(112.557641)), int(np.round(216.879357))],
         [int(np.round(241.587131)), int(np.round(433.758713)), int(np.round(90.595174)), int(np.round(161.973190))],
         [int(np.round(691.817694)), int(np.round(387.088472)), int(np.round(82.359249)), int(np.round(178.445040))],
         [int(np.round(782.412869)), int(np.round(527.099196)), int(np.round(85.104558)), int(np.round(175.699732))]
        ]
for box in boxes:
    img_array = cv2.rectangle(img_array,(box[0],box[1]),(box[0]+box[2],box[1]+box[3]),(0,255,0),10)
plt.imshow(img_array)

Like image classification, face detections is a supervised training task. Here, the inputs are the images and the outputs are the bounding box coordinates.

MXNet lets us convert images into a binary Record format that makes I/O blazing fast, and we'll use this format to train our model in the next step.

## Train

Like image classification, face detections is a supervised training task. Here, the inputs are the images and the outputs are the bounding box coordinates.

MXNet lets us convert images into a binary Record format that makes I/O blazing fast, and we'll use this format to train our model in the next step.

Now that we understand our data and what the task is, we can launch a training job on SageMaker. To do that, we need to create our SageMaker session object and our execution role with the following methods that inherit the credentials/ARN from the notebook environment:

In [ ]:
sagemaker_session = sagemaker.Session()
role = get_execution_role()

Next, we need to put the data set on S3 so SageMaker can pull the data set onto the instances it spins up for training.

The data has been pre-loaded into S3, so we'll use it's address as the location of our inputs.

In [ ]:
inputs = "s3://sagemaker-obj-det-demo/ssd-data"

Next, we can easily launch a training job by defining a custom <tt>MXNet</tt> estimator, through which we specify custom training code and hyperparameters, as well as the type and number of instances we wish to train on at once. Tuning hyperparameters and launching distributed training jobs is as simple as changing these arguments below.

In [ ]:
net_sagemaker = MXNet("mxnet_ssd_face_detection.py", 
                  role=role, 
                  train_instance_count=1, 
                  train_instance_type="ml.p2.xlarge",
                  sagemaker_session=sagemaker_session,
                  hyperparameters={
                                 'batch_size': 8, 
                                 'epochs': 1, 
                                 'learning_rate': 0.004, 
                                 'num_gpus': 1
                                  })                 

With a single click, we can launch a training job, distributed or otherwise, and SageMaker completely handles the resource management to do so. 

When launching a training job, SageMaker:
* Creates an S3 bucket to store training script <tt>mxnet_ssd_face_detection.py</tt>
* Launches an EC2 instance(s) for training
* Pulls down data in S3 and training script
* Runs the training loop defined in the <tt>train_fn</tt> method in our training script, <tt>mxnet_ssd_face_detection.py</tt>
* The model begins to train, logging the training loss and metrics every 200 iterations. We are only training for one epoch, but it is enough to see that the loss is indeed being minimized

In [ ]:
net_sagemaker.fit(inputs)

## Deploy

We've trained our model here for a single epoch in the interest of time, but in reality we would want to run this overnight, which we've already done. After this, we'd make some modifications to our model to convert it from training format to inference.

We've done this already, and will use SageMaker to deploy this model to an endpoint that will run inference on data requests.

In [ ]:
mxnet_model = MXNetModel(model_data="s3://sagemaker-obj-det-demo/model.tar.gz",
                         role=role, entry_point="mxnet_ssd_face_detection.py",
                        py_version='py3')

As before, the actual deployment will take only one click; SageMaker takes care of launching the instances and the server itself.

In [ ]:
predictor = mxnet_model.deploy(instance_type="ml.c4.xlarge", initial_instance_count=1)
predictor.serializer = None
predictor.deserializer = None

Once we've deployed the inference endpoint, we can start calling it by sending requests with serialized image data.

In [ ]:
test_file = '19_Couple_Couple_19_187.jpg'
with open(test_file, mode='rb') as f: 
    img_buffer = f.read()
response = predictor.predict(img_buffer.encode('base64'))

The response we recieve is a series of possible face detections:
<tt>[id, score, xmin, ymin, xmax, ymax]</tt>

We only have one class so <tt>id</tt> will always be zero. However, we can set a threshold for what level of confidence we want to have to include face detections as valid.

In [ ]:
def plot_prediction(response, test_file, thresh=0.5):
    dets = pickle.loads(response)
    dets = dets[0]
    img = cv2.imread(test_file)
    img[:, :, (0, 1, 2)] = img[:, :, (2, 1, 0)]
    classes = ['face']
    plt.imshow(img)
    height = img.shape[0]
    width = img.shape[1]
    colors = dict()
    for i in range(dets.shape[0]):
        cls_id = int(dets[i, 0])
        if cls_id >= 0:
            score = dets[i, 1]
            if score > thresh:
                if cls_id not in colors:
                    colors[cls_id] = (random.random(), random.random(), random.random())
                xmin = int(dets[i, 2] * width)
                ymin = int(dets[i, 3] * height)
                xmax = int(dets[i, 4] * width)
                ymax = int(dets[i, 5] * height)
                rect = plt.Rectangle((xmin, ymin), xmax - xmin,
                                     ymax - ymin, fill=False,
                                     edgecolor=colors[cls_id],
                                     linewidth=3.5)
                plt.gca().add_patch(rect)
                class_name = str(cls_id)
                if classes and len(classes) > cls_id:
                    class_name = classes[cls_id]
                plt.gca().text(xmin, ymin - 2,
                                '{:s} {:.3f}'.format(class_name, score),
                                bbox=dict(facecolor=colors[cls_id], alpha=0.5),
                                fontsize=12, color='white')
    plt.show()

The plot shows the bounding box around the identified face, as well as the confidence level of the prediction.

In [ ]:
plot_prediction(response, test_file)